In [1]:
#run as seconda shell
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


# Read the CSV file
data = pd.read_csv('DL_info.csv')

# Define the input shape of the images
input_shape = (512, 512, 1)

# Preprocess the data
def preprocess_data(data, input_shape, base_dir):
    images = []
    masks = []
    read= 0 
    unread  =0 
    
    for _, row in data.iterrows():
        
        patient_index = row['Patient_index']
        study_index = row['Study_index']
        series_index = row['Series_ID']
        slice_index = row['Key_slice_index']
        bounding_boxes = eval(row['Bounding_boxes'])
        if(patient_index>1000):continue
        le = len(str(slice_index))
        p_name = row['File_name']
        file_name = p_name[len(p_name)-7:]
        # Construct the file path based on the naming convention
       
        p_name = p_name[:len(p_name)-le-6]
        file_path = os.path.join(base_dir, p_name, file_name)
        
        # Load the 16-bit grayscale image
        try :

            image = cv2.imread(file_path, cv2.IMREAD_UNCHANGED)
            image = cv2.resize(image, input_shape[:2])
            read = read +1
        except:
  
            unread = unread+1
            data = data[data['Key_slice_index'] != slice_index ]
            continue
        # Apply windowing
        window_center = 40
        window_width = 80
        min_value = window_center - window_width // 2
        max_value = window_center + window_width // 2
        image = np.clip(image, min_value, max_value)
        
        # Normalize the image to 0-1 range
        image = (image - min_value) / (max_value - min_value)
        images.append(image)
        
        # Create the mask from bounding boxes
        mask = np.zeros(input_shape[:2], dtype=np.uint8)
        bbox  = bounding_boxes 
        
        # bbox = list(map(float, bbox.split(',')))
        x1, y1, x2, y2 = int(float(bbox[0])) , int(float(bbox[1])), int(float(bbox[2])), int(float(bbox[3]))
       
        mask[y1:y2, x1:x2] = 1
        masks.append(mask)
    
    images = np.expand_dims(np.array(images), axis=-1)
    masks = np.expand_dims(np.array(masks), axis=-1)
    print("read:", read, "unread :",unread)
    return images, masks

# Set the base directory where the images are stored
base_dir = 'all/Images_png'

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Preprocess the training and testing data
train_images, train_masks = preprocess_data(train_data, input_shape, base_dir)
test_images, test_masks = preprocess_data(test_data, input_shape, base_dir)



2024-03-22 05:14:00.165601: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 05:14:01.012682: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[ WARN:0@2.972] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000910_11_0/441.png'): can't open/read file: check file path/integrity
[ WARN:0@2.985] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000083_09_0/376.png'): can't open/read file: check file path/integrity
[ WARN:0@3.007] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000332_07_0/136.png'): can't open/read file: check file path/integrity
[ WARN:0@3.014] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000299_05_0/191.png'): can't open/read file: ch

read: 5198 unread : 3331


[ WARN:0@38.432] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000238_05_0/150.png'): can't open/read file: check file path/integrity
[ WARN:0@38.448] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000187_15_0/344.png'): can't open/read file: check file path/integrity
[ WARN:0@38.453] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000569_15_0/101.png'): can't open/read file: check file path/integrity
[ WARN:0@38.458] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000235_02_0/119.png'): can't open/read file: check file path/integrity
[ WARN:0@38.464] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000552_05_0/122.png'): can't open/read file: check file path/integrity
[ WARN:0@38.470] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000642_02_0/139.png'): can't open/read file: check file path/integrity
[ WARN:0@38.479] global loadsave.cpp:248 findDecoder imread_('all/Images_png/000669_01_0/440.png'): can't open/r

read: 1233 unread : 852


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, Dropout, MaxPooling2D, UpSampling2D, Concatenate

class PatchExtract(tf.keras.layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super(PatchExtract, self).__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding='VALID'
        )
        return patches

def patch_extract(inputs, patch_size):
    return PatchExtract(patch_size)(inputs)

def patch_embedding(inputs, num_patches, projection_dim):
    x = tf.keras.layers.Dense(projection_dim)(inputs)
    x = tf.keras.layers.Reshape((num_patches, projection_dim))(x)
    return x

@tf.function
def swin_transformer_block(inputs, num_heads, window_size, shift_size):
    x = tf.keras.layers.LayerNormalization(epsilon=1e-5)(inputs)
    x = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1] // num_heads)(x, x)
    x = tf.keras.layers.Add()([x, inputs])

    x = tf.keras.layers.LayerNormalization(epsilon=1e-5)(x)
    x = tf.keras.layers.Dense(x.shape[-1], activation='gelu')(x)
    x = tf.keras.layers.Dense(inputs.shape[-1], activation=None)(x)
    x = tf.keras.layers.Add()([x, inputs])

    return x

def ssformer(input_shape):
    inputs = Input(shape=input_shape)

    # Patch extraction and embedding
    patch_size = 4
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
    projection_dim = 96
    x = patch_extract(inputs, patch_size)
    x = patch_embedding(x, num_patches, projection_dim)

    # Swin Transformer blocks
    num_heads = 3
    window_size = 7
    shift_size = window_size // 2
    for i in range(4):
        x = swin_transformer_block(x, num_heads, window_size, shift_size)

    # Decoder path
    x = tf.keras.layers.Reshape((input_shape[0] // patch_size, input_shape[1] // patch_size, projection_dim))(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, 3, activation='relu', padding='same')(x)
    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(32, 3, activation='relu', padding='same')(x)
    x = Conv2D(1, 1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Set memory growth for GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Rest of your code

# Clear the TensorFlow session after training or inference
tf.keras.backend.clear_session()

2024-03-22 05:14:44.600379: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-22 05:14:44.711536: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Define the ssformer model
model = ssformer(input_shape)
model.summary()


ValueError: A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

In [ ]:
print(len(train_images),len(train_masks))


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
batch_size = 2
epochs = 50
model.fit(train_images, train_masks, batch_size=batch_size, epochs=epochs, validation_data=(test_images, test_masks))

# Evaluate the model on the testing set
loss, accuracy = model.evaluate(test_images, test_masks)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

# Make predictions on new images
new_image_path = 'path/to/new/16bit/image.png'
new_image = cv2.imread(new_image_path, cv2.IMREAD_UNCHANGED)
new_image = cv2.resize(new_image, input_shape[:2])

# Apply windowing to the new image
window_center = 40
window_width = 80
min_value = window_center - window_width // 2
max_value = window_center + window_width // 2
new_image = np.clip(new_image, min_value, max_value)
new_image = (new_image - min_value) / (max_value - min_value)

new_image = np.expand_dims(new_image, axis=0)
new_image = np.expand_dims(new_image, axis=-1)

prediction = model.predict(new_image)
predicted_mask = (prediction > 0.5).astype(np.uint8)

# Visualize the predicted mask
cv2.imshow('Predicted Mask', predicted_mask[0, :, :, 0])
cv2.waitKey(0)
cv2.destroyAllWindows()

2024-03-22 05:13:02.838213: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5450498048 exceeds 10% of free system memory.
